#GA algorithm

In [ ]:
pip install pyirt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pyirt]


In [ ]:
pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 5.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from deap import base, creator, tools, algorithms
import random


## import

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
from deap import base, creator, tools, algorithms
import random

In [ ]:
# 역채점 변환기 사용 0~ 4 로써 5점 척도
def inverse_scale(df):
    return (df - 4) * (-1)

In [ ]:
# 데이터 로드 및 전처리
df = pd.read_csv('/content/drive/MyDrive/학부연구생/C프로젝트/df.csv')
df = df.dropna()
# 1. 문항 이름과 번호 매칭 딕셔너리 생성
# 열 이름을 고유 번호로 매칭
original_columns = df.columns
column_to_index = {col: idx for idx, col in enumerate(original_columns)}
index_to_column = {idx: col for idx, col in enumerate(original_columns)}

# 무응답 데이터 삭제
zero_index = df[(df == 0).sum(axis=1) != 0].index
df.drop(zero_index, inplace=True)
df.reset_index(drop=True, inplace=True)

#역채점 문항 정채점으로
역채점문항 = ['EXT2','EXT4','EXT6','EXT8','EXT10','EST2','EST4','AGR1','AGR3','AGR5','AGR7','CSN2','CSN4','CSN6','CSN8','OPN2','OPN4','OPN6']
df[역채점문항] = inverse_scale(df[역채점문항])



In [ ]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
df.columns

Index(['EXT1', 'EXT2', 'EXT3', 'EXT4', 'EXT5', 'EXT6', 'EXT7', 'EXT8', 'EXT9',
       'EXT10', 'EST1', 'EST2', 'EST3', 'EST4', 'EST5', 'EST6', 'EST7', 'EST8',
       'EST9', 'EST10', 'AGR1', 'AGR2', 'AGR3', 'AGR4', 'AGR5', 'AGR6', 'AGR7',
       'AGR8', 'AGR9', 'AGR10', 'CSN1', 'CSN2', 'CSN3', 'CSN4', 'CSN5', 'CSN6',
       'CSN7', 'CSN8', 'CSN9', 'CSN10', 'OPN1', 'OPN2', 'OPN3', 'OPN4', 'OPN5',
       'OPN6', 'OPN7', 'OPN8', 'OPN9', 'OPN10'],
      dtype='object')

In [ ]:
df.to_csv("processed_df_revision.csv", index=False)

OSError: Cannot save file into a non-existent directory: '/content/drive/MyDrive/학부연구생/C프로젝트/C프로젝트 수정데이터'

## df정리

In [ ]:
from deap import creator

# 기존 정의 삭제
if 'FitnessMax' in creator.__dict__:
    del creator.FitnessMax
if 'Individual' in creator.__dict__:
    del creator.Individual

# 다시 정의
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

In [ ]:
import pandas as pd

# 데이터 로드 및 전처리
df = pd.read_csv('/content/drive/MyDrive/학부연구생/C프로젝트/df.csv')
original_columns = df.columns
column_to_index = {col: idx for idx, col in enumerate(original_columns)}
index_to_column = {idx: col for idx, col in enumerate(original_columns)}
df = df.dropna()
# 1. 문항 이름과 번호 매칭 딕셔너리 생성
# === 2. IRT 정보 불러오기 후 매핑 ===
irt_df = pd.read_csv('/content/drive/MyDrive/학부연구생/C프로젝트/C프로젝트 수정 데이터/item_info_with_discrimination.csv')

irt_df['index'] = irt_df['Item'].map(column_to_index)

# NaN이 포함되어 있으면 경고
if irt_df['index'].isnull().any():
    print("⚠️ 매핑되지 않은 문항 있음:", irt_df[irt_df['index'].isnull()]['Item'].tolist())

# === 3. 딕셔너리 구성 ===
irt_info_dict = {
    int(row['index']): {
        'info': row['Info_theta0'],
        'a_mean': np.mean([row[f'a{i}'] for i in range(1, 6)])
    }
    for _, row in irt_df.iterrows()
    if not pd.isna(row['index'])
}

# 열 이름을 고유 번호로 매칭
original_columns = df.columns
column_to_index = {col: idx for idx, col in enumerate(original_columns)}
index_to_column = {idx: col for idx, col in enumerate(original_columns)}

# 무응답 데이터 삭제
zero_index = df[(df == 0).sum(axis=1) != 0].index
df.drop(zero_index, inplace=True)
df.reset_index(drop=True, inplace=True)

#역채점 문항 정채점으로
역채점문항 = ['EXT2','EXT4','EXT6','EXT8','EXT10','EST2','EST4','AGR1','AGR3','AGR5','AGR7','CSN2','CSN4','CSN6','CSN8','OPN2','OPN4','OPN6']
df[역채점문항] = inverse_scale(df[역채점문항])

# 2. 문항 이름을 번호로 바꿔서 데이터프레임 처리
df.columns = range(df.shape[1])  # 열 인덱스를 0, 1, 2, ... 로 변환

# 원래 심리 척도 데이터 및 점수 계산
num_items = len(df.columns)  # 원래 문항 수
num_samples = len(df)  # 샘플 수
data = df.values
original_scores = data.sum(axis=1)  # 원래 설문 점수

# 요인별 문항을 10개씩 5개 요인으로 나누기
factor_1_items = list(range(0, 10))    # 1~10번 문항은 요인 1
factor_2_items = list(range(10, 20))   # 11~20번 문항은 요인 2
factor_3_items = list(range(20, 30))   # 21~30번 문항은 요인 3
factor_4_items = list(range(30, 40))   # 31~40번 문항은 요인 4
factor_5_items = list(range(40, 50))   # 41~50번 문항은 요인 5

# 각 요인별로 선택할 문항 수 설정
target_num_items_per_factor = 5  # 각 요인에서 선택할 문항 수

# 크론바흐 알파 계산 함수
def cronbach_alpha(items_scores):
    items_scores = np.array(items_scores)
    item_variances = items_scores.var(axis=1, ddof=1)
    total_variance = items_scores.sum(axis=0).var(ddof=1)
    n_items = items_scores.shape[0]
    return n_items / (n_items - 1) * (1 - item_variances.sum() / total_variance)

# 유전자형 및 적합도 함수 정의
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)  # 유전자형은 0 또는 1로 이루어진 리스트
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=num_items)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evaluate(individual):
    selected_items = np.array(individual, dtype=bool)

    # 요인별 선택 문항 확인
    selected_per_factor = [
        [i for i in f if selected_items[i]]
        for f in [factor_1_items, factor_2_items, factor_3_items, factor_4_items, factor_5_items]
    ]
    if any(len(group) != target_num_items_per_factor for group in selected_per_factor):
        return -1000.,

    # 선택된 문항 데이터
    reduced_data = data[:, selected_items]
    reduced_scores = reduced_data.sum(axis=1)

    # 상관 + 신뢰도
    correlation = pearsonr(reduced_scores, original_scores)[0]
    alpha = cronbach_alpha(reduced_data)

    # IRT 정보 평균 계산
    selected_indices = np.where(selected_items)[0]
    info_avg = np.mean([irt_info_dict[int(i)]['info'] for i in selected_indices])
    a_avg = np.mean([irt_info_dict[int(i)]['a_mean'] for i in selected_indices])

    # 총 적합도 계산 (가중치는 필요시 조절 가능)
    fitness = correlation + alpha + info_avg + a_avg

    return fitness,
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)  # 두 지점을 교차하는 교차 연산
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)  # 비트를 반전시키는 돌연변이 연산
toolbox.register("select", tools.selTournament, tournsize=3)  # 토너먼트 선택

# 유전 알고리즘 실행
def main():
    random.seed(42)
    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)  # 최고 적합도 개체 저장
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    # 유전 알고리즘 실행
    algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.1, ngen=100, stats=stats, halloffame=hof, verbose=True)

    return pop, stats, hof

if __name__ == "__main__":
    pop, stats, hof = main()

    # 3. 최종 결과: 번호를 원래 문항 이름으로 변환하여 출력
    best_individual = hof[0]
    selected_items = np.where(np.array(best_individual) == 1)[0]  # 선택된 문항의 인덱스
    selected_column_names = [index_to_column[idx] for idx in selected_items]  # 번호를 원래 이름으로 변환

    print("Best individual (selected column names): ", selected_column_names)
    print("With fitness: ", best_individual.fitness.values[0])

/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals	avg  	std	min  	max  
0  	300   	-1000	0  	-1000	-1000
1  	211   	-1000	0  	-1000	-1000


KeyError: 0

In [ ]:
import random
import numpy as np

RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
def main():
    random.seed(RANDOM_SEED)
    np.random.seed(RANDOM_SEED)

    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.1, ngen=100,
                        stats=stats, halloffame=hof, verbose=True)

    return pop, stats, hof

In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import random
from deap import base, creator, tools, algorithms
from scipy.stats import pearsonr

# 역채점 함수 정의
def inverse_scale(df_subset):
    return 6 - df_subset

# 경로 설정
data_path = '/content/drive/MyDrive/학부연구생/C프로젝트/df.csv'
irt_path = '/content/drive/MyDrive/학부연구생/C프로젝트/C프로젝트 수정 데이터/item_info_with_discrimination.csv'

# 1. 데이터 불러오기 및 전처리
df = pd.read_csv(data_path).dropna()
original_columns = df.columns
column_to_index = {col: idx for idx, col in enumerate(original_columns)}
index_to_column = {idx: col for idx, col in enumerate(original_columns)}
df = df[(df != 0).all(axis=1)].reset_index(drop=True)

# 역채점 문항 처리
reverse_items = ['EXT2','EXT4','EXT6','EXT8','EXT10','EST2','EST4','AGR1','AGR3','AGR5','AGR7',
                 'CSN2','CSN4','CSN6','CSN8','OPN2','OPN4','OPN6']
df[reverse_items] = inverse_scale(df[reverse_items])

# 열 번호로 변환
df.columns = range(df.shape[1])
num_items = df.shape[1]
data = df.values
original_scores = data.sum(axis=1)

# 요인별 문항
factor_items = [list(range(i, i+10)) for i in range(0, 50, 10)]
target_num_items_per_factor = 5

# 크론바흐 알파 계산
def cronbach_alpha(items_scores):
    items_scores = np.array(items_scores)
    item_variances = items_scores.var(axis=1, ddof=1)
    total_variance = items_scores.sum(axis=0).var(ddof=1)
    n_items = items_scores.shape[0]
    return n_items / (n_items - 1) * (1 - item_variances.sum() / total_variance)

# IRT 정보 로드 및 매핑
irt_df = pd.read_csv(irt_path)
irt_df['index'] = irt_df['Item'].map(column_to_index)
irt_info_dict = {
    int(row['index']): {
        'info': row['Info_theta0'],
        'a_mean': np.mean([row[f'a{i}'] for i in range(1, 6)])
    }
    for _, row in irt_df.iterrows()
}

# DEAP 설정
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=num_items)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# 적합도 함수
def evaluate(individual):
    selected_items = np.array(individual, dtype=bool)
    selected_per_factor = [[i for i in f if selected_items[i]] for f in factor_items]
    if any(len(g) != target_num_items_per_factor for g in selected_per_factor):
        return -1000.,

    reduced_data = data[:, selected_items]
    reduced_scores = reduced_data.sum(axis=1)
    correlation = pearsonr(reduced_scores, original_scores)[0]
    alpha = cronbach_alpha(reduced_data)

    selected_indices = np.where(selected_items)[0]
    try:
        info_avg = np.mean([irt_info_dict[int(i)]['info'] for i in selected_indices])
        a_avg = np.mean([irt_info_dict[int(i)]['a_mean'] for i in selected_indices])
    except KeyError:
        return -1000.,

    return correlation + alpha + info_avg + a_avg,

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

# 실행 함수
def main():
    random.seed(42)
    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.1, ngen=100,
                        stats=stats, halloffame=hof, verbose=True)
    return pop, stats, hof

# 실행
pop, stats, hof = main()
best_individual = hof[0]
selected_items = np.where(np.array(best_individual) == 1)[0]
selected_column_names = [index_to_column[idx] for idx in selected_items]

print("Best individual (selected column names):", selected_column_names)
print("With fitness:", best_individual.fitness.values[0])

/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals	avg  	std	min  	max  
0  	300   	-1000	0  	-1000	-1000
1  	211   	-1000	0  	-1000	-1000
2  	201   	-1000	0  	-1000	-1000
3  	216   	-1000	0  	-1000	-1000
4  	196   	-1000	0  	-1000	-1000
5  	221   	-1000	0  	-1000	-1000
6  	213   	-1000	0  	-1000	-1000
7  	233   	-996.661	57.7362	-1000	1.6913
8  	209   	-993.322	81.5139	-1000	1.6913
9  	213   	-983.305	128.235	-1000	1.6913
10 	232   	-996.661	57.7351	-1000	1.67238
11 	213   	-1000   	0      	-1000	-1000  
12 	206   	-1000   	0      	-1000	-1000  
13 	209   	-1000   	0      	-1000	-1000  
14 	219   	-1000   	0      	-1000	-1000  
15 	213   	-1000   	0      	-1000	-1000  
16 	231   	-996.661	57.7365	-1000	1.69598
17 	198   	-993.322	81.5151	-1000	1.69598
18 	231   	-996.661	57.7365	-1000	1.69598
19 	229   	-996.661	57.736 	-1000	1.68772
20 	200   	-1000   	0      	-1000	-1000  
21 	207   	-1000   	0      	-1000	-1000  
22 	210   	-1000   	0      	-1000	-1000  
23 	220   	-1000   	0      	-1000	-1000  
24 	216   	-996.661	57.73

In [ ]:
selected_column_names

['EXT1',
 'EXT3',
 'EXT4',
 'EXT5',
 'EXT7',
 'EST2',
 'EST5',
 'EST7',
 'EST8',
 'EST9',
 'EST10',
 'AGR1',
 'AGR4',
 'AGR8',
 'AGR9',
 'AGR10',
 'CSN1',
 'CSN5',
 'CSN6',
 'CSN7',
 'CSN10',
 'OPN2',
 'OPN3',
 'OPN5',
 'OPN8']

In [ ]:
selected_column_names

['EXT1',
 'EXT3',
 'EXT4',
 'EXT5',
 'EXT7',
 'EST2',
 'EST5',
 'EST7',
 'EST8',
 'EST9',
 'EST10',
 'AGR1',
 'AGR4',
 'AGR8',
 'AGR9',
 'AGR10',
 'CSN1',
 'CSN5',
 'CSN6',
 'CSN7',
 'CSN10',
 'OPN2',
 'OPN3',
 'OPN5',
 'OPN8']

In [ ]:
Ga_result_index = selected_column_names
pd.DataFrame(Ga_result_index).to_csv('/content/drive/MyDrive/학부연구생/C프로젝트/C프로젝트 수정 데이터/Ga_result_index.csv')